In [36]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import confusion_matrix, classification_report, r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import BaggingClassifier, GradientBoostingClassifier, StackingClassifier, BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor, StackingRegressor
from catboost import CatBoostClassifier, CatBoostRegressor

# Композиции модели для задачи классификации

## Подготовка данных

In [3]:
balance_scale_df = pd.read_csv('../data/balance_scale_preprocessed.csv')
balance_scale_df.head()

,Class_Name,Left-Weight,Left-Distance,Right-Weight,Right-Distance
0,B,1,1,1,1
1,R,1,1,1,2
2,R,1,1,1,3
3,R,1,1,1,4
4,R,1,1,1,5


In [4]:
X = balance_scale_df.iloc[:, 1:5].values
print(X)

[[1 1 1 1]
 [1 1 1 2]
 [1 1 1 3]
 ...
 [5 5 5 3]
 [5 5 5 4]
 [5 5 5 5]]


In [5]:
y = balance_scale_df.iloc[:, 0].values
print(y)

['B' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'L' 'B' 'R' 'R' 'R' 'B' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'L' 'L' 'B' 'R'
 'R' 'L' 'R' 'R' 'R' 'R' 'B' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'L' 'L' 'L' 'B' 'R' 'L' 'B' 'R' 'R' 'R' 'L' 'R' 'R' 'R' 'R'
 'B' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'L' 'L' 'L' 'L' 'B' 'L' 'L' 'R'
 'R' 'R' 'L' 'R' 'R' 'R' 'R' 'L' 'R' 'R' 'R' 'R' 'B' 'R' 'R' 'R' 'R' 'L'
 'B' 'R' 'R' 'R' 'B' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R'
 'R' 'R' 'R' 'R' 'R' 'R' 'L' 'L' 'L' 'B' 'R' 'L' 'B' 'R' 'R' 'R' 'L' 'R'
 'R' 'R' 'R' 'B' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'R' 'L' 'L' 'L' 'L' 'L'
 'L' 'L' 'B' 'R' 'R' 'L' 'B' 'R' 'R' 'R' 'L' 'R' 'R' 'R' 'R' 'L' 'R' 'R'
 'R' 'R' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'B' 'R' 'L' 'L' 'R' 'R' 'R' 'L'
 'B' 'R' 'R' 'R' 'L' 'R' 'R' 'R' 'R' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L' 'L'
 'B' 'L' 'L' 'L' 'R' 'R' 'L' 'L' 'R' 'R' 'R' 'L' 'B

Разбиваем исходные данные на обучающую и контрольную выборки. Обучающая выборка составляет 80% от объема исходных данных, а контрольная составляет, соответственно, 20%.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

Объявляем *StandardScaler* для нормализации данных.

In [7]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Обучение модели

В качестве базового алгоритма используем классификатор дерева решений.

In [8]:
%%time
parameters_dtc = {'max_depth': np.arange(5,16)}
dtc = DecisionTreeClassifier()
dtc_base = GridSearchCV(dtc, parameters_dtc).fit(X_train, y_train)
dtc_base.best_params_

CPU times: user 96.1 ms, sys: 2.98 ms, total: 99.1 ms
Wall time: 103 ms


{'max_depth': 6}

In [9]:
y_pred = dtc_base.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 2  4  6]
 [ 3 47  7]
 [ 7  5 44]]
              precision    recall  f1-score   support

           B       0.17      0.17      0.17        12
           L       0.84      0.82      0.83        57
           R       0.77      0.79      0.78        56

    accuracy                           0.74       125
   macro avg       0.59      0.59      0.59       125
weighted avg       0.74      0.74      0.74       125



Данные результаты возможно улучшить.

## Обучение композиций

###  Модель BaggingClassifier

Объявим гиперпараметры для всех моделей.

In [10]:
parameters_ensemble = {'n_estimators': np.arange(20,101,20),
                       'max_features': np.arange(3,24,10)}

In [11]:
%%time
bag = BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=dtc_base.best_params_['max_depth']))
model = GridSearchCV(bag, parameters_ensemble).fit(X_train, y_train)
model.best_params_

CPU times: user 2.46 s, sys: 46.1 ms, total: 2.51 s
Wall time: 2.57 s


{'max_features': 3, 'n_estimators': 80}

In [12]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 0  6  6]
 [ 0 52  5]
 [ 0  3 53]]
              precision    recall  f1-score   support

           B       0.00      0.00      0.00        12
           L       0.85      0.91      0.88        57
           R       0.83      0.95      0.88        56

    accuracy                           0.84       125
   macro avg       0.56      0.62      0.59       125
weighted avg       0.76      0.84      0.80       125



Точность в первом случае ухудшилась, в двух случаях немного улучшилась, на поиск параметров ушло гораздо больше времени.

### Модель GradientBoostingClassifier

In [13]:
%%time
gbc = GradientBoostingClassifier()
model = GridSearchCV(gbc, parameters_ensemble).fit(X_train, y_train)
model.best_params_

CPU times: user 3.5 s, sys: 31.9 ms, total: 3.54 s
Wall time: 3.62 s


{'max_features': 3, 'n_estimators': 80}

In [14]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 1  7  4]
 [ 0 54  3]
 [ 2  0 54]]
              precision    recall  f1-score   support

           B       0.33      0.08      0.13        12
           L       0.89      0.95      0.92        57
           R       0.89      0.96      0.92        56

    accuracy                           0.87       125
   macro avg       0.70      0.66      0.66       125
weighted avg       0.83      0.87      0.84       125



Точность во всех трех случаях улучшилась, но на поиск параметров ушло гораздо больше времени.

### Модель StackingClassifier

In [15]:
%%time
model = StackingClassifier(estimators=[('bag',bag), ('gbc',gbc)],
                           final_estimator=dtc_base).fit(X_train, y_train)

CPU times: user 1.56 s, sys: 22.1 ms, total: 1.58 s
Wall time: 1.63 s


In [16]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[ 6  4  2]
 [ 2 55  0]
 [ 3  5 48]]
              precision    recall  f1-score   support

           B       0.55      0.50      0.52        12
           L       0.86      0.96      0.91        57
           R       0.96      0.86      0.91        56

    accuracy                           0.87       125
   macro avg       0.79      0.77      0.78       125
weighted avg       0.87      0.87      0.87       125



Точность во всех трех случаях улучшилась, значительно она возросла для первого класса. Времени затрачивается меньше, чем при бэггинге и бустинге.

### Модель CatBoostClassifier

In [21]:
%%time
cbc = CatBoostClassifier(learning_rate=0.15).fit(X_train, y_train)
y_pred = cbc.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

0:	learn: 0.9909202	total: 55ms	remaining: 54.9s
1:	learn: 0.9030186	total: 58.7ms	remaining: 29.3s
2:	learn: 0.8262202	total: 59.8ms	remaining: 19.9s
3:	learn: 0.7594976	total: 60.5ms	remaining: 15.1s
4:	learn: 0.7109993	total: 61.3ms	remaining: 12.2s
5:	learn: 0.6667973	total: 62.1ms	remaining: 10.3s
6:	learn: 0.6346901	total: 62.7ms	remaining: 8.89s
7:	learn: 0.5981446	total: 63.4ms	remaining: 7.86s
8:	learn: 0.5712711	total: 64.5ms	remaining: 7.1s
9:	learn: 0.5424479	total: 65.3ms	remaining: 6.47s
10:	learn: 0.5191893	total: 66.1ms	remaining: 5.95s
11:	learn: 0.5052640	total: 66.7ms	remaining: 5.49s
12:	learn: 0.4867772	total: 67.7ms	remaining: 5.14s
13:	learn: 0.4704453	total: 68.5ms	remaining: 4.83s
14:	learn: 0.4533200	total: 69.5ms	remaining: 4.57s
15:	learn: 0.4398964	total: 70.2ms	remaining: 4.32s
16:	learn: 0.4261949	total: 75.8ms	remaining: 4.38s
17:	learn: 0.4135577	total: 77.3ms	remaining: 4.21s
18:	learn: 0.4080435	total: 77.9ms	remaining: 4.02s
19:	learn: 0.3982419	tota

160:	learn: 0.1009708	total: 223ms	remaining: 1.16s
161:	learn: 0.1005406	total: 226ms	remaining: 1.17s
162:	learn: 0.1000746	total: 230ms	remaining: 1.18s
163:	learn: 0.0996680	total: 231ms	remaining: 1.18s
164:	learn: 0.0991216	total: 232ms	remaining: 1.17s
165:	learn: 0.0984659	total: 233ms	remaining: 1.17s
166:	learn: 0.0978719	total: 234ms	remaining: 1.17s
167:	learn: 0.0974063	total: 234ms	remaining: 1.16s
168:	learn: 0.0969991	total: 237ms	remaining: 1.16s
169:	learn: 0.0965373	total: 238ms	remaining: 1.16s
170:	learn: 0.0958091	total: 238ms	remaining: 1.16s
171:	learn: 0.0954501	total: 239ms	remaining: 1.15s
172:	learn: 0.0949342	total: 241ms	remaining: 1.15s
173:	learn: 0.0945826	total: 243ms	remaining: 1.15s
174:	learn: 0.0940804	total: 244ms	remaining: 1.15s
175:	learn: 0.0937489	total: 244ms	remaining: 1.14s
176:	learn: 0.0933509	total: 245ms	remaining: 1.14s
177:	learn: 0.0928935	total: 246ms	remaining: 1.14s
178:	learn: 0.0924698	total: 251ms	remaining: 1.15s
179:	learn: 

517:	learn: 0.0320197	total: 567ms	remaining: 527ms
518:	learn: 0.0319296	total: 568ms	remaining: 526ms
519:	learn: 0.0318686	total: 569ms	remaining: 525ms
520:	learn: 0.0318132	total: 570ms	remaining: 524ms
521:	learn: 0.0317294	total: 571ms	remaining: 523ms
522:	learn: 0.0316692	total: 572ms	remaining: 521ms
523:	learn: 0.0316048	total: 573ms	remaining: 520ms
524:	learn: 0.0315277	total: 573ms	remaining: 519ms
525:	learn: 0.0314452	total: 574ms	remaining: 517ms
526:	learn: 0.0313246	total: 575ms	remaining: 516ms
527:	learn: 0.0312795	total: 576ms	remaining: 515ms
528:	learn: 0.0312160	total: 577ms	remaining: 514ms
529:	learn: 0.0311664	total: 578ms	remaining: 512ms
530:	learn: 0.0311159	total: 578ms	remaining: 511ms
531:	learn: 0.0310617	total: 579ms	remaining: 510ms
532:	learn: 0.0310028	total: 580ms	remaining: 508ms
533:	learn: 0.0309622	total: 581ms	remaining: 507ms
534:	learn: 0.0308909	total: 582ms	remaining: 506ms
535:	learn: 0.0308375	total: 583ms	remaining: 504ms
536:	learn: 

742:	learn: 0.0215467	total: 739ms	remaining: 256ms
743:	learn: 0.0215275	total: 740ms	remaining: 255ms
744:	learn: 0.0214983	total: 742ms	remaining: 254ms
745:	learn: 0.0214557	total: 743ms	remaining: 253ms
746:	learn: 0.0214249	total: 744ms	remaining: 252ms
747:	learn: 0.0213941	total: 745ms	remaining: 251ms
748:	learn: 0.0213607	total: 745ms	remaining: 250ms
749:	learn: 0.0213318	total: 746ms	remaining: 249ms
750:	learn: 0.0213025	total: 747ms	remaining: 248ms
751:	learn: 0.0212800	total: 748ms	remaining: 247ms
752:	learn: 0.0212408	total: 748ms	remaining: 246ms
753:	learn: 0.0212206	total: 749ms	remaining: 244ms
754:	learn: 0.0211967	total: 750ms	remaining: 243ms
755:	learn: 0.0211728	total: 751ms	remaining: 242ms
756:	learn: 0.0211410	total: 751ms	remaining: 241ms
757:	learn: 0.0211122	total: 752ms	remaining: 240ms
758:	learn: 0.0210758	total: 753ms	remaining: 239ms
759:	learn: 0.0210514	total: 754ms	remaining: 238ms
760:	learn: 0.0210200	total: 755ms	remaining: 237ms
761:	learn: 

905:	learn: 0.0173277	total: 873ms	remaining: 90.6ms
906:	learn: 0.0173019	total: 875ms	remaining: 89.7ms
907:	learn: 0.0172814	total: 893ms	remaining: 90.5ms
908:	learn: 0.0172572	total: 894ms	remaining: 89.5ms
909:	learn: 0.0172410	total: 898ms	remaining: 88.8ms
910:	learn: 0.0172264	total: 899ms	remaining: 87.8ms
911:	learn: 0.0172063	total: 901ms	remaining: 86.9ms
912:	learn: 0.0171860	total: 908ms	remaining: 86.5ms
913:	learn: 0.0171663	total: 909ms	remaining: 85.6ms
914:	learn: 0.0171508	total: 911ms	remaining: 84.6ms
915:	learn: 0.0171366	total: 912ms	remaining: 83.7ms
916:	learn: 0.0171162	total: 914ms	remaining: 82.7ms
917:	learn: 0.0170956	total: 915ms	remaining: 81.8ms
918:	learn: 0.0170801	total: 919ms	remaining: 81ms
919:	learn: 0.0170579	total: 921ms	remaining: 80ms
920:	learn: 0.0170412	total: 922ms	remaining: 79.1ms
921:	learn: 0.0170243	total: 923ms	remaining: 78.1ms
922:	learn: 0.0170052	total: 925ms	remaining: 77.1ms
923:	learn: 0.0169896	total: 926ms	remaining: 76.1

По сравнению с базовым алгоритмом точность значительно улучшилась. Данная модель показывает лучший результат по скорости.

Стекинг показывает наилучший результат.

# Композиции модели для задачи регрессии

## Подготовка данных

In [45]:
machine_df = pd.read_csv('../data/machine_preprocessed.csv')
machine_df.head()

,Vendor_Name,Model_Name,MYCT,MMIN,MMAX,CACH,CHMIN,CHMAX,PRP,ERP
0,adviser,32/60,125,256,6000,256,16,128,198,199
1,amdahl,470v/7,29,8000,32000,32,8,32,269,253
2,amdahl,470v/7a,29,8000,32000,32,8,32,220,253
3,amdahl,470v/7b,29,8000,32000,32,8,32,172,253
4,amdahl,470v/7c,29,8000,16000,32,8,16,132,132


In [46]:
X = machine_df.iloc[:,2:9].values
print(X)

[[  125   256  6000 ...    16   128   198]
 [   29  8000 32000 ...     8    32   269]
 [   29  8000 32000 ...     8    32   220]
 ...
 [  125  2000  8000 ...     2    14    52]
 [  480   512  8000 ...     0     0    67]
 [  480  1000  4000 ...     0     0    45]]


In [47]:
y = machine_df.iloc[:,-1].values
print(y)

[ 199  253  253  253  132  290  381  381  749 1238   23   24   70  117
   15   64   23   29   22  124   35   39   40   45   28   21   28   22
   28   27  102  102   74   74  138  136   23   29   44   30   41   74
   74   74   54   41   18   28   36   38   34   19   72   36   30   56
   42   34   34   34   34   34   19   75  113  157   18   20   28   33
   47   54   20   23   25   52   27   50   18   53   23   30   73   20
   25   28   29   32  175   57  181  181   32   82  171  361  350  220
  113   15   21   35   18   20   20   28   45   18   17   26   28   28
   31   31   42   76   76   26   59   65  101  116   18   20   20   30
   44   44   82   82  128   37   46   46   80   88   88   33   46   29
   53   53   41   86   95  107  117  119  120   48  126  266  270  426
  151  267  603   19   21   26   35   41   47   62   78   80   80  142
  281  190   21   25   67   24   24   64   25   20   29   43   53   19
   22   31   41   47   99   67   81  149  183  275  382   56  182  227
  341 

Разбиваем исходные данные на обучающую и контрольную выборки. Обучающая выборка составляет 80% от объема исходных данных, а контрольная составляет, соответственно, 20%.

In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

Объявляем *StandardScaler* для нормализации данных.

In [60]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Обучение модели

В качестве базового алгоритма используем регрессор дерева решений.

In [61]:
%%time
parameters_dtr = {'max_depth': np.arange(5,16,1)}
dtr = DecisionTreeRegressor().fit(X_train, y_train)
dtr_base = GridSearchCV(dtr, parameters_dtr).fit(X_train, y_train)
dtr_base.best_params_

CPU times: user 99.2 ms, sys: 3.21 ms, total: 102 ms
Wall time: 112 ms


{'max_depth': 11}

In [62]:
y_pred = dtr_base.predict(X_test)
print(f"Коэффициент детерминации: {r2_score(y_test, y_pred)}")
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test,y_pred, squared=False)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')

Коэффициент детерминации: 0.9108358142960737
MSE: 2898.9173280423283
RMSE: 53.84159477617958
MAE: 19.28174603174603


Значение коэффициента детерминации достаточное неплохое, однако ошибки потенциально могут быть уменьшены.

## Обучение композиций

###  Модель BaggingRegressor

In [63]:
%%time
br = BaggingRegressor(base_estimator=DecisionTreeRegressor(max_depth=dtr_base.best_params_['max_depth']))
model = GridSearchCV(br, parameters_ensemble).fit(X_train, y_train)
model.best_params_

CPU times: user 2.18 s, sys: 44.8 ms, total: 2.22 s
Wall time: 2.29 s


{'max_features': 3, 'n_estimators': 20}

In [64]:
y_pred = model.predict(X_test)
print(f"Коэффициент детерминации: {r2_score(y_test, y_pred)}")
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test,y_pred, squared=False)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')

Коэффициент детерминации: 0.9917790492935762
MSE: 267.2805932974999
RMSE: 16.348718399235455
MAE: 10.376546710564568


После применения бэггинга значение коэффициента детерминации значчительно улучшилось, количество ошибок уменьшилось. Время выполнения значительно возросло.

###  Модель GradientBoostingRegressor

In [65]:
%%time
gbr = GradientBoostingRegressor()
model = GridSearchCV(gbr, parameters_ensemble).fit(X_train, y_train)
model.best_params_

CPU times: user 472 ms, sys: 6.28 ms, total: 478 ms
Wall time: 489 ms


{'max_features': 3, 'n_estimators': 80}

In [66]:
y_pred = model.predict(X_test)
print(f"Коэффициент детерминации: {r2_score(y_test, y_pred)}")
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test,y_pred, squared=False)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')

Коэффициент детерминации: 0.9737473618498171
MSE: 853.5291052070819
RMSE: 29.215220437420662
MAE: 10.997637715399858


Точность увеличилась, количество ошибок уменьшилось по сравнению с базовым алгоритмом. Время выполнения значительно меньше, чем у бэггинга.

### Модель StackingRegressor

In [71]:
%%time
model = StackingRegressor(estimators=[('br',br), ('gbr',gbr)],
                           final_estimator=dtr_base).fit(X_train, y_train)

CPU times: user 351 ms, sys: 5.72 ms, total: 357 ms
Wall time: 367 ms


In [72]:
y_pred = model.predict(X_test)
print(f"Коэффициент детерминации: {r2_score(y_test, y_pred)}")
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test,y_pred, squared=False)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')

Коэффициент детерминации: 0.9591564846702704
MSE: 1327.9095568783068
RMSE: 36.44049336765773
MAE: 16.581349206349206


Точность увеличилась, количество ошибок уменьшилось по сравнению с базовым алгоритмом. Время выполнения значительно меньше, чем у бэггинга.

### Модель CatBoostRegressor

In [73]:
%%time
cbr = CatBoostRegressor(learning_rate=0.15).fit(X_train, y_train)
y_pred = model.predict(X_test)
print(f"Коэффициент детерминации: {r2_score(y_test, y_pred)}")
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {mean_squared_error(y_test,y_pred, squared=False)}')
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')

0:	learn: 135.3338926	total: 1.51ms	remaining: 1.51s
1:	learn: 124.7108927	total: 3.1ms	remaining: 1.55s
2:	learn: 116.9110115	total: 4.31ms	remaining: 1.43s
3:	learn: 110.5850227	total: 5.3ms	remaining: 1.32s
4:	learn: 102.6936850	total: 6.28ms	remaining: 1.25s
5:	learn: 96.1142498	total: 7.42ms	remaining: 1.23s
6:	learn: 90.1330000	total: 8.28ms	remaining: 1.17s
7:	learn: 84.4916268	total: 9.24ms	remaining: 1.15s
8:	learn: 79.1468246	total: 11.1ms	remaining: 1.23s
9:	learn: 74.4961107	total: 12.1ms	remaining: 1.2s
10:	learn: 70.0478341	total: 13.1ms	remaining: 1.18s
11:	learn: 65.9648753	total: 14.1ms	remaining: 1.16s
12:	learn: 64.0922134	total: 15.4ms	remaining: 1.17s
13:	learn: 61.6209526	total: 16.5ms	remaining: 1.16s
14:	learn: 58.4792536	total: 17.8ms	remaining: 1.17s
15:	learn: 55.3534399	total: 19.1ms	remaining: 1.18s
16:	learn: 51.7255722	total: 20.1ms	remaining: 1.16s
17:	learn: 49.8519099	total: 21ms	remaining: 1.15s
18:	learn: 48.4722477	total: 21.7ms	remaining: 1.12s
19:

364:	learn: 0.4158762	total: 347ms	remaining: 604ms
365:	learn: 0.4118087	total: 349ms	remaining: 605ms
366:	learn: 0.4074966	total: 350ms	remaining: 604ms
367:	learn: 0.4011882	total: 351ms	remaining: 603ms
368:	learn: 0.3999704	total: 352ms	remaining: 602ms
369:	learn: 0.3971280	total: 353ms	remaining: 601ms
370:	learn: 0.3942701	total: 354ms	remaining: 600ms
371:	learn: 0.3904805	total: 355ms	remaining: 599ms
372:	learn: 0.3855794	total: 356ms	remaining: 598ms
373:	learn: 0.3824319	total: 357ms	remaining: 597ms
374:	learn: 0.3789330	total: 358ms	remaining: 596ms
375:	learn: 0.3762133	total: 358ms	remaining: 595ms
376:	learn: 0.3740732	total: 360ms	remaining: 594ms
377:	learn: 0.3737987	total: 361ms	remaining: 593ms
378:	learn: 0.3708039	total: 362ms	remaining: 592ms
379:	learn: 0.3660165	total: 362ms	remaining: 591ms
380:	learn: 0.3650616	total: 363ms	remaining: 590ms
381:	learn: 0.3647765	total: 364ms	remaining: 589ms
382:	learn: 0.3618345	total: 365ms	remaining: 588ms
383:	learn: 

540:	learn: 0.1339729	total: 518ms	remaining: 439ms
541:	learn: 0.1332105	total: 531ms	remaining: 449ms
542:	learn: 0.1328084	total: 532ms	remaining: 448ms
543:	learn: 0.1323401	total: 533ms	remaining: 447ms
544:	learn: 0.1318603	total: 534ms	remaining: 446ms
545:	learn: 0.1308864	total: 538ms	remaining: 447ms
546:	learn: 0.1293860	total: 539ms	remaining: 446ms
547:	learn: 0.1291093	total: 540ms	remaining: 445ms
548:	learn: 0.1287006	total: 540ms	remaining: 444ms
549:	learn: 0.1275556	total: 541ms	remaining: 443ms
550:	learn: 0.1267706	total: 548ms	remaining: 446ms
551:	learn: 0.1264941	total: 548ms	remaining: 445ms
552:	learn: 0.1256888	total: 549ms	remaining: 444ms
553:	learn: 0.1254888	total: 550ms	remaining: 443ms
554:	learn: 0.1247137	total: 551ms	remaining: 442ms
555:	learn: 0.1231194	total: 552ms	remaining: 441ms
556:	learn: 0.1221769	total: 553ms	remaining: 440ms
557:	learn: 0.1219750	total: 555ms	remaining: 439ms
558:	learn: 0.1203665	total: 557ms	remaining: 439ms
559:	learn: 

733:	learn: 0.0456945	total: 689ms	remaining: 250ms
734:	learn: 0.0453976	total: 691ms	remaining: 249ms
735:	learn: 0.0450418	total: 692ms	remaining: 248ms
736:	learn: 0.0450266	total: 692ms	remaining: 247ms
737:	learn: 0.0449313	total: 693ms	remaining: 246ms
738:	learn: 0.0445681	total: 694ms	remaining: 245ms
739:	learn: 0.0444264	total: 695ms	remaining: 244ms
740:	learn: 0.0442001	total: 696ms	remaining: 243ms
741:	learn: 0.0437612	total: 697ms	remaining: 242ms
742:	learn: 0.0434575	total: 698ms	remaining: 241ms
743:	learn: 0.0431716	total: 699ms	remaining: 240ms
744:	learn: 0.0429339	total: 700ms	remaining: 240ms
745:	learn: 0.0427736	total: 701ms	remaining: 239ms
746:	learn: 0.0424904	total: 702ms	remaining: 238ms
747:	learn: 0.0419585	total: 703ms	remaining: 237ms
748:	learn: 0.0418366	total: 705ms	remaining: 236ms
749:	learn: 0.0416349	total: 706ms	remaining: 235ms
750:	learn: 0.0415158	total: 708ms	remaining: 235ms
751:	learn: 0.0414160	total: 708ms	remaining: 234ms
752:	learn: 

951:	learn: 0.0165209	total: 862ms	remaining: 43.4ms
952:	learn: 0.0164690	total: 863ms	remaining: 42.5ms
953:	learn: 0.0163425	total: 864ms	remaining: 41.6ms
954:	learn: 0.0162467	total: 867ms	remaining: 40.8ms
955:	learn: 0.0161966	total: 868ms	remaining: 40ms
956:	learn: 0.0161278	total: 869ms	remaining: 39.1ms
957:	learn: 0.0160465	total: 870ms	remaining: 38.1ms
958:	learn: 0.0160406	total: 872ms	remaining: 37.3ms
959:	learn: 0.0158420	total: 875ms	remaining: 36.5ms
960:	learn: 0.0157801	total: 885ms	remaining: 35.9ms
961:	learn: 0.0156777	total: 897ms	remaining: 35.4ms
962:	learn: 0.0156734	total: 899ms	remaining: 34.5ms
963:	learn: 0.0154798	total: 900ms	remaining: 33.6ms
964:	learn: 0.0153848	total: 902ms	remaining: 32.7ms
965:	learn: 0.0151759	total: 909ms	remaining: 32ms
966:	learn: 0.0150881	total: 911ms	remaining: 31.1ms
967:	learn: 0.0150238	total: 915ms	remaining: 30.2ms
968:	learn: 0.0149805	total: 916ms	remaining: 29.3ms
969:	learn: 0.0149590	total: 918ms	remaining: 28.4

Точность увеличилась, количество ошибок уменьшилось по сравнению с базовым алгоритмом.

Стекинг показывает неплохой результат но отстаёт от остальных композиций. Лучшей композицией оказался бэггинг.

**Вывод**: Использование композиций алгоритмов дает повышает качество моделей, однако на обучение моделей уходит гораздо больше времени.